In [1]:
from azureml.core import Workspace, Experiment, Run
from azureml.core.model import Model
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import joblib
import os

In [2]:
# Load MNIST dataset
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
y = y.astype(int)  # Convert target to integer

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train MLP model
mlp = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', solver='adam', max_iter=20, random_state=42)
mlp.fit(X_train, y_train)

# Evaluate model
y_pred = mlp.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Save model
model_path = "mnist_mlp_model.pkl"
joblib.dump(mlp, model_path)

Test Accuracy: 0.9754


['mnist_mlp_model.pkl']

In [3]:
# Azure ML Setup
ws = Workspace.from_config()
model = Model.register(workspace=ws,
                        model_path=model_path,
                        model_name="mnist-mlp",
                        tags={"accuracy": f"{accuracy:.4f}"},
                        description="MLP model for MNIST classification")

print(f"Model registered: {model.name} | Version: {model.version}")


Registering model mnist-mlp
Model registered: mnist-mlp | Version: 1
